In [ ]:
url = "http://192.168.3.6:8080"
graph_name = "network"
user_id = "65cf8f2ca2605827790739034ae9e58f7efb3968"

In [ ]:
from langgraph_sdk import get_client, get_sync_client

client = get_client(url=url)
sync_client = get_sync_client(url=url)

- 创建Thread （一个对话，表示用户的对话）

In [ ]:
thread = sync_client.threads.create(metadata={"user_id": user_id})

- Cron job (Unsupport in Standlone Container)

In [ ]:
cron_job = await client.crons.create_for_thread(
    thread["thread_id"],
    graph_name,
    schedule="*/5 * * * *",
    input={"messages": [{"role": "user", "content": "What time is it?"}]},
)

In [ ]:
await client.crons.delete(cron_job["cron_id"])

- 发送消息

In [ ]:
config = {"configurable": {"thread_id": thread["thread_id"]}}

input = {
    "messages": {
        "type": "human",
        "content": "把这个地址上的bnb和usdt加入到我的仓位中。0x25Dd4FD54a3c0C70cD780000D9a0eE90d93B8E85",
    },
    "wallet_is_connected": True,
    "chain_id": 56,
    "wallet_address": "0x25Dd4FD54a3c0C70cD780000D9a0eE90d93B8E85",
    "time_zone": "Asia/Shanghai",
    "user_id": user_id,
}
chunks = client.runs.stream(
    thread_id=thread["thread_id"], assistant_id=graph_name, input=input
)

- 打印所有chunk，包含所有处理过程事件（数据，例如，工具返回值）

In [ ]:
async for chunk in chunks:
    print(f"Event:{chunk.event}")
    print(chunk.data)
    print("=" * 100)

- 查询用户所有的对话

In [ ]:
threads = await client.threads.search(metadata={"user_id": user_id}, limit=100)
for thread in threads:
    print(thread.values())